In [1]:
from __future__ import print_function

%matplotlib inline
import numpy as np 
import scipy.misc # to visualize only  
import pandas as pd
import matplotlib.pyplot as plt
import string
# !pip install tensorflow
import tensorflow as tf
print(tf.test.gpu_device_name())
# !pip install opencv-python
import cv2
import pandas as pd

# !pip install keras
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Concatenate, Input, concatenate, Lambda
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop, Adagrad, Nadam
from math import exp
import warnings
import math
import random
import itertools
# !pip install sklearn
from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.layers import Layer
from keras import activations
from keras import utils
from keras.models import Model
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

from keras.models import model_from_json

# !pip install xgboost
import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import GridSearchCV

#cnn with xgboost
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

# LOAD LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

import sys
import numpy as np
import cv2 as cv
import sklearn.metrics as sklm

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model, load_model
from keras.datasets import mnist

from keras import backend as K
img_dim_ordering = 'tf'
K.set_image_dim_ordering(img_dim_ordering)

train_images = pd.read_pickle('./train_images.pkl')
train_labels = pd.read_csv('./train_labels.csv')
test_images = pd.read_pickle('./test_images.pkl')

Using TensorFlow backend.


# Preprocessing

In [2]:
# Preprocessing
x = train_images
y = train_labels['Category'].values
x = x.reshape(-1, 64, 64)
y = y.reshape(-1, 1) 
x = np.int64(x)
y = np.int64(y)

test = test_images
test = test.reshape(-1, 64, 64)
test = np.int64(x) 

def preprocess(image):
    connectivity = 4
    img = image.astype(np.uint8)
    _,thresh = cv.threshold(img,254,255,cv.THRESH_BINARY)

    
    img_blurred = cv.blur(img,(3,3))

    _,thresh_blurred = cv.threshold(img_blurred,205,255,cv.THRESH_BINARY)    

    output = cv.connectedComponentsWithStats(thresh_blurred, connectivity, cv.CV_32S)
    n_components = output[0] - 1
    stats = output[2][1:]
    
    # Too many high intensity parts, go without blur
    if n_components >= 4:
        output = cv.connectedComponentsWithStats(thresh, connectivity, cv.CV_32S)
        n_components = output[0] - 1
        stats = output[2][1:]
    
     
    largest_component = np.array([])
    largest_area = 0
    largest_length = 0
    largest_height = 0
    for i in range(n_components):
        
        left = stats[i][0]
        top = stats[i][1]
        length = stats[i][2]
        height = stats[i][3]
        if max(length,height) >=32:
            continue
        area = max(length,height) ** 2 # for square bounding box
        
        if(area > largest_area):
            largest_area = area
            largest_length = length
            largest_height = height
            largest_component = thresh[top:height+top, left:left+length]

            
    result = np.zeros([64,64],dtype=np.uint8)
    result[0:largest_height,0:largest_length] = result[0:largest_height,0:largest_length] + largest_component    
    result = result[:32,:32]
    return result


x1 = np.zeros([x.shape[0],32,32],dtype=np.uint8)
test1 = np.zeros([x.shape[0],32,32],dtype=np.uint8)
for i in range(x.shape[0]):
    x1[i] = preprocess(x[i])

In [9]:
x.shape

(40000, 64, 64)

In [10]:
num_classes = 10
img_dim = 32

x_train, x_val, y_train, y_val = train_test_split(x1, y, test_size=0.2)

x_train = x_train.reshape(x_train.shape[0], img_dim, img_dim, 1)
x_val = x_val.reshape(x_val.shape[0], img_dim, img_dim, 1)
input_shape = (img_dim, img_dim, 1)


x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255


y_train = utils.to_categorical(y_train, num_classes)
y_val = utils.to_categorical(y_val, num_classes)

# Modified VGG-16

In [12]:
num_classes = 10
batch_size = 32
epochs = 10

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 20:
        lr *= 1e-3
    elif epoch > 15:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(32,32,1)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Dropout(0.3))

model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Dropout(0.3))

model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Dropout(0.3))


model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Dropout(0.3))


model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()


lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=4,min_lr=0.5e-9)
callbacks = [lr_reducer, lr_scheduler]


history = model.fit(x_train, y_train, 
                              batch_size=batch_size, 
                              epochs = epochs, 
                    callbacks=callbacks,
                    validation_data = (x_val,y_val))
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14 (ZeroPaddi (None, 34, 34, 1)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 32)        320       
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 34, 34, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 34, 34, 32)        128       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 32)        0         
__________

32000/32000 [==============================] - 301s 9ms/step - loss: 0.3782 - acc: 0.9140 - val_loss: 0.3215 - val_acc: 0.9273
Epoch 6/10
Learning rate:  0.001
32000/32000 [==============================] - 301s 9ms/step - loss: 0.3514 - acc: 0.9225 - val_loss: 0.2900 - val_acc: 0.9350
Epoch 7/10
Learning rate:  0.001
32000/32000 [==============================] - 301s 9ms/step - loss: 0.3335 - acc: 0.9262 - val_loss: 0.2969 - val_acc: 0.9325
Epoch 8/10
Learning rate:  0.001
32000/32000 [==============================] - 300s 9ms/step - loss: 0.3163 - acc: 0.9298 - val_loss: 0.2982 - val_acc: 0.9323
Epoch 9/10
Learning rate:  0.001
32000/32000 [==============================] - 288s 9ms/step - loss: 0.3015 - acc: 0.9318 - val_loss: 0.2540 - val_acc: 0.9401
Epoch 10/10
Learning rate:  0.001
32000/32000 [==============================] - 289s 9ms/step - loss: 0.3030 - acc: 0.9333 - val_loss: 0.2811 - val_acc: 0.9314
Test loss: 0.2811066158413887
Test accuracy: 0.931375


In [6]:
model.save("YMVGG.h5")

# Make predictions

In [7]:
x_test = test_images.reshape(-1, 64, 64) # reshape 
x_test = np.uint8(x_test)
processed = []
    
for i in range(len(x_test)):
    ret,thresh = cv2.threshold(x_test[i],254,255,cv2.THRESH_BINARY)
    processed.append(thresh)
processed = np.array(processed)
data = processed.reshape(processed.shape[0], 64, 64, 1)


data = data.astype('float32')
data /= 255
predict = model.predict_classes(data)
my_submission = pd.DataFrame({'Id': np.arange(len(predict)), 'Category': predict})
my_submission.to_csv('submission_ymvgg.csv', index=False)